In [7]:
# Instalar paquetes necesarios
!pip install flake8 pylint coverage

import json
import os

# Definir la ruta de archivos en el entorno de Colab
DATA_PATH = "/content/"

# Crear y guardar data_handler.py (Clase Base)
with open("data_handler.py", "w") as f:
    f.write("""
\"\"\"Módulo base para carga de datos.\"\"\"

import json
import os

class DataHandler:
    \"\"\"Clase base para manejar datos en archivos JSON.\"\"\"

    @classmethod
    def load_from_file(cls):
        \"\"\"Carga los datos desde un archivo JSON.\"\"\"
        if not os.path.exists(cls.FILE_PATH):
            return []
        with open(cls.FILE_PATH, "r", encoding="utf-8") as file:
            try:
                data = json.load(file)
                return [cls(**item) for item in data]
            except (json.JSONDecodeError, KeyError):
                print(f"Error al cargar los datos de {cls.__name__}.")
                return []
""")

# Crear y guardar hotels.py
with open("hotels.py", "w") as f:
    f.write("""
\"\"\"Módulo para gestionar hoteles.\"\"\"

import json
import os
from data_handler import DataHandler

DATA_PATH = "/content/"


class Hotel(DataHandler):
    \"\"\"Clase para representar un hotel.\"\"\"

    FILE_PATH = os.path.join(DATA_PATH, "hotels.json")

    def __init__(self, hotel_id, name, location, rooms):
        \"\"\"Inicializa un hotel con ID, nombre, ubicación y habitaciones.\"\"\"
        self.hotel_id = hotel_id
        self.name = name
        self.location = location
        self.rooms = rooms

    def to_dict(self):
        \"\"\"Convierte el objeto a un diccionario.\"\"\"
        return {
            "hotel_id": self.hotel_id,
            "name": self.name,
            "location": self.location,
            "rooms": self.rooms
        }

    @classmethod
    def save_to_file(cls, hotels):
        \"\"\"Guarda la lista de hoteles en un archivo JSON.\"\"\"
        with open(cls.FILE_PATH, "w", encoding="utf-8") as file:
            json.dump(
                [hotel.to_dict() for hotel in hotels],
                file,
                indent=4
            )
""")

# Crear y guardar customers.py
with open("customers.py", "w") as f:
    f.write("""
\"\"\"Módulo para gestionar clientes.\"\"\"

import json
import os
from data_handler import DataHandler

DATA_PATH = "/content/"


class Customer(DataHandler):
    \"\"\"Clase para representar un cliente.\"\"\"

    FILE_PATH = os.path.join(DATA_PATH, "customers.json")

    def __init__(self, customer_id, name, email):
        \"\"\"Inicializa un cliente con ID, nombre y correo electrónico.\"\"\"
        self.customer_id = customer_id
        self.name = name
        self.email = email

    def to_dict(self):
        \"\"\"Convierte el objeto a un diccionario.\"\"\"
        return {
            "customer_id": self.customer_id,
            "name": self.name,
            "email": self.email
        }

    @classmethod
    def save_to_file(cls, customers):
        \"\"\"Guarda la lista de clientes en un archivo JSON.\"\"\"
        with open(cls.FILE_PATH, "w", encoding="utf-8") as file:
            json.dump(
                [customer.to_dict() for customer in customers],
                file,
                indent=4
            )
""")

# Crear y guardar reservations.py con la corrección de E501
with open("reservations.py", "w") as f:
    f.write("""
\"\"\"Módulo para gestionar reservas.\"\"\"

import json
import os
from data_handler import DataHandler

DATA_PATH = "/content/"


class Reservation(DataHandler):
    \"\"\"Clase para representar una reserva.\"\"\"

    FILE_PATH = os.path.join(DATA_PATH, "reservations.json")

    def __init__(self, reservation_id, hotel_id, customer_id, room_number):
        \"\"\"Inicializa una reserva con ID, hotel, cliente y número de habitación.\"\"\"
        self.reservation_id = reservation_id
        self.hotel_id = hotel_id
        self.customer_id = customer_id
        self.room_number = room_number

    def to_dict(self):
        \"\"\"Convierte el objeto a un diccionario.\"\"\"
        return {
            "reservation_id": self.reservation_id,
            "hotel_id": self.hotel_id,
            "customer_id": self.customer_id,
            "room_number": self.room_number
        }

    @classmethod
    def save_to_file(cls, reservations):
        \"\"\"Guarda la lista de reservas en un archivo JSON.\"\"\"
        with open(cls.FILE_PATH, "w", encoding="utf-8") as file:
            json.dump(
                [res.to_dict() for res in reservations],
                file,
                indent=4
            )
""")

# Crear y guardar test_system.py
with open("test_system.py", "w") as f:
    f.write("""
import unittest
from hotels import Hotel
from customers import Customer
from reservations import Reservation


class TestSystem(unittest.TestCase):
    '''Pruebas unitarias para el sistema de reservas.'''

    def setUp(self):
        '''Configura datos iniciales para cada prueba.'''
        self.hotel = Hotel("1", "Luxury Inn", "New York", [101, 102, 103])
        self.customer = Customer("C1", "John Doe", "john@example.com")
        self.reservation = Reservation("R1", "1", "C1", 101)

    def test_create_hotel(self):
        self.assertEqual(self.hotel.name, "Luxury Inn")

    def test_create_customer(self):
        self.assertEqual(self.customer.name, "John Doe")

    def test_create_reservation(self):
        self.assertEqual(self.reservation.room_number, 101)

    def test_save_and_load_hotel(self):
        '''Prueba guardar y cargar datos de hoteles.'''
        Hotel.save_to_file([self.hotel])
        loaded_hotels = Hotel.load_from_file()
        self.assertEqual(loaded_hotels[0].name, "Luxury Inn")

    def test_save_and_load_customer(self):
        '''Prueba guardar y cargar datos de clientes.'''
        Customer.save_to_file([self.customer])
        loaded_customers = Customer.load_from_file()
        self.assertEqual(loaded_customers[0].name, "John Doe")

    def test_save_and_load_reservation(self):
        '''Prueba guardar y cargar datos de reservas.'''
        Reservation.save_to_file([self.reservation])
        loaded_reservations = Reservation.load_from_file()
        self.assertEqual(loaded_reservations[0].room_number, 101)


if __name__ == "__main__":
    unittest.main()
""")

# Ejecutar pruebas unitarias
!python test_system.py

# Verificar cobertura de código
!coverage run test_system.py
!coverage report -m

# Revisar formato con flake8 y pylint
!flake8 hotels.py customers.py reservations.py
!pylint hotels.py customers.py reservations.py


......
----------------------------------------------------------------------
Ran 6 tests in 0.001s

OK
......
----------------------------------------------------------------------
Ran 6 tests in 0.002s

OK
Name              Stmts   Miss  Cover   Missing
-----------------------------------------------
customers.py         16      0   100%
data_handler.py      14      4    71%   14, 19-21
hotels.py            17      0   100%
reservations.py      17      0   100%
test_system.py       29      0   100%
-----------------------------------------------
TOTAL                93      4    96%
reservations.py:17:80: E501 line too long (83 > 79 characters)

-------------------------------------------------------------------
Your code has been rated at 10.00/10 (previous run: 9.79/10, +0.21)

